# Title of Database: Wall-Following navigation task with mobile robot SCITOS-G5

The data were collected as the SCITOS G5 navigates through the room following the wall in a clockwise
direction, for 4 rounds. To navigate, the robot uses 24 ultrasound sensors arranged circularly around its "waist". 
The numbering of the ultrasound sensors starts at the front of the robot and increases in clockwise direction.

In [77]:
# modules
from keras.layers import Input, Dense, Dropout
from keras.models import Model
from keras.datasets import mnist
from keras.models import Sequential, load_model
from keras.optimizers import RMSprop
from keras.callbacks import TensorBoard
from __future__ import print_function
from keras.utils import plot_model
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from sklearn import preprocessing
from keras import layers

import keras
import matplotlib.pyplot as plt
import numpy as np
import math
import pydot
import graphviz
import pandas as pd

In [51]:
# import
data = pd.read_csv('data/sensor_readings_24.csv', sep=",", header=None)

In [52]:
data.head()

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,0.438,0.498,3.625,3.645,5.0,2.918,5.0,2.351,2.332,2.643,...,0.593,0.502,0.493,0.504,0.445,0.431,0.444,0.440,0.429,Slight-Right-Turn
1,0.438,0.498,3.625,3.648,5.0,2.918,5.0,2.637,2.332,2.649,...,0.592,0.502,0.493,0.504,0.449,0.431,0.444,0.443,0.429,Slight-Right-Turn
2,0.438,0.498,3.625,3.629,5.0,2.918,5.0,2.637,2.334,2.643,...,0.593,0.502,0.493,0.504,0.449,0.431,0.444,0.446,0.429,Slight-Right-Turn
3,0.437,0.501,3.625,3.626,5.0,2.918,5.0,2.353,2.334,2.642,...,0.593,0.502,0.493,0.504,0.449,0.431,0.444,0.444,0.429,Slight-Right-Turn
4,0.438,0.498,3.626,3.629,5.0,2.918,5.0,2.640,2.334,2.639,...,0.592,0.502,0.493,0.504,0.449,0.431,0.444,0.441,0.429,Slight-Right-Turn


The data is not normalized and the classes are given as strings. We are going to change that.

In [53]:
mapping = {key: value for (key, value) in zip(data[24].unique(), range(len(data[24].unique())))}
data.replace({24:mapping}, inplace=True)

In [140]:
data[24].head(10)

0    0
1    0
2    0
3    0
4    0
5    0
6    0
7    0
8    1
9    1
Name: 24, dtype: int64

Take a random sample of 90% of the rows from the dataframe. To ensure reproducability the `random_state` variable is set. The other 10% are placed aside for validation after training. The last column is the class column and is stored in the y variables respectively.

In [55]:
data_train = data.sample(frac=0.9, random_state=42)
data_val = data.drop(data_train.index)

df_x_train = data_train.iloc[:,:-1]
df_y_train = data_train.iloc[:,-1]

df_x_val = data_val.iloc[:,:-1]
df_y_val = data_val.iloc[:,-1]

Normalization

In [56]:
x_train = df_x_train.values
x_train = (x_train - x_train.min()) / (x_train.max() - x_train.min())
y_train = df_y_train.values

x_val = df_x_val.values
x_val = (x_val - x_val.min()) / (x_val.max() - x_val.min())
y_val = df_y_val.values

Make useful categorical variables out of the single column data by one-hot encoding it.

In [57]:
y_train = keras.utils.to_categorical(y_train, 4)
y_val = keras.utils.to_categorical(y_val, 4)

# Train Neural Net

The dimension of the hidden layers are set arbitrarily but some runs have shown that 30 is a good number. The `input_dim` Variable is set to 24 because initially there are 24 features.

In [58]:
# constants
num_classes = 4
input_dim = x_train.shape[1]
hidden1_dim = 30
hidden2_dim = 30

In [59]:
result = []

for i in range(1,4):
    model = Sequential()
    model.add(Dense(hidden1_dim, activation='relu', input_shape=(input_dim,)))
    model.add(Dense(hidden2_dim, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(loss='categorical_crossentropy',
                  optimizer=RMSprop(),
                  metrics=['accuracy'])

    model.fit(x_train, y_train, 
              batch_size=24,
              epochs=100,
              verbose=0,
              shuffle=True,
              validation_split=0.1)
    score = model.evaluate(x_val, y_val)[1]
    result.append(score)
    print(result)
print(np.mean(score))

546/546 [==============================] - 0s 34us/step
[0.93040293040293043]
546/546 [==============================] - 0s 64us/step
[0.93040293040293043, 0.9285714285714286]
546/546 [==============================] - 0s 32us/step
[0.93040293040293043, 0.9285714285714286, 0.9432234432234432]
0.943223443223


## SingleAutoencoder with classifier

In [60]:
encoder_dim1 = 8

In [90]:
main_input = Input(shape=(input_dim,), dtype='float32', name='main_input')

encoding_layer = Dense(encoder_dim1, activation='relu')
encoding_layer_output = encoding_layer(main_input)
decoding_layer_output = Dense(input_dim, activation='sigmoid', name='decoder_output')(encoding_layer_output)

x = Dense(hidden1_dim, activation='relu')(encoding_layer_output)
x = Dense(hidden2_dim, activation='relu')(x)

classifier_output = Dense(num_classes, activation='softmax', name='main_output')(x)

auto_classifier = Model(inputs=main_input, outputs=[classifier_output, decoding_layer_output])

auto_classifier.compile(optimizer=RMSprop(),
                        loss={'main_output': 'categorical_crossentropy', 'decoder_output': 'mean_squared_error'},
                        loss_weights={'main_output': 1., 'decoder_output': 1.},
                        metrics=['accuracy'])

auto_classifier.fit({'main_input': x_train},
                    {'main_output': y_train, 'decoder_output': x_train},
                    epochs=100, 
                    batch_size=24,
                    shuffle=True,
                    validation_split=0.1,
                    verbose=0)

score = auto_classifier.evaluate(x=x_val, y=[y_val, x_val], verbose=1)
print(score[3])

546/546 [==============================] - 0s 83us/step
0.880952380952


In [118]:
encoding_weights = encoding_layer.get_weights()
sum_of_weights = {index: item.sum() for (index, item) in enumerate(encoding_weights[0])}

weights = sum_of_weights
features = []
for i in range(encoder_dim1):
    max_key = max(weights, key=lambda key: weights[key])
    features.append(max_key)
    del weights[max_key]
    
print(features)   

[19, 18, 3, 15, 8, 10, 23, 9]


## Check the neural net performance with those new selected features

In [133]:
x_train_selected = np.array([x[features] for x in x_train])
x_val_selected = np.array([x[features] for x in x_val])

In [135]:
input_dim = x_train_selected.shape[1]
hidden1_dim = 16
hidden2_dim = 16

In [136]:
result = []

for i in range(1,4):
    model_new = Sequential()
    model_new.add(Dense(hidden1_dim, activation='relu', input_shape=(input_dim,)))
    model_new.add(Dense(hidden2_dim, activation='relu'))
    model_new.add(Dense(num_classes, activation='softmax'))

    model_new.compile(loss='categorical_crossentropy',
                  optimizer=RMSprop(),
                  metrics=['accuracy'])

    model_new.fit(x_train_selected, y_train, 
              batch_size=24,
              epochs=100,
              verbose=0,
              shuffle=True,
              validation_split=0.1)
    score = model_new.evaluate(x_val_selected, y_val)[1]
    result.append(score)
    print(result)
print(np.mean(score))

546/546 [==============================] - 0s 62us/step
[0.81135531135531136]
546/546 [==============================] - 0s 39us/step
[0.81135531135531136, 0.80219780219780223]
546/546 [==============================] - 0s 63us/step
[0.81135531135531136, 0.80219780219780223, 0.82783882783882778]
0.827838827839
